In [1]:
#Importing Libraries
import pandas as pd
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Read Product-category File
json_file_path = r'C:\Users\Lenovo\Downloads\products.json'

In [3]:
#Read Products File
products_online = pd.read_excel(r'C:\Users\Lenovo\Downloads\Stroller Harshit Sheet.xlsx')

In [4]:
products_online.head()

,Internal ID,BRAND,Name,Store Display Name,All-Terrain tires (K0bby),Transit option (Bus tie-down),Tilt Adjustment,Recline Adjustment Upper,Recline Adjustment Lower,Product Weight,Weight capacity,Seat depth Lower,Seat depth Upper,Seat width Lower,Seat width Upper,Reversible Seating
0,361,Adaptive Star,ASENDEAVOUR,Axiom Endeavour Mobility Push Chair,1,0,0,0,30,43.0,250.0,15.0,18.0,15.0,21.0,0
1,362,Adaptive Star,ASIMPROV,Axiom Improv Mobility Push Chair,1,0,0,0,30,35.0,200.0,15.0,15.0,15.0,18.0,0
2,363,Adaptive Star,ASLASSEN,Axiom Lassen Mobility Push Chair,1,0,0,0,30,35.0,250.0,15.0,18.0,15.0,21.0,0
3,48671,Adaptive Star,ASPHOENIX,Axiom Phoenix Mobility Push Chair,1,0,0,0,140,30.0,200.0,16.0,16.0,18.0,18.0,0
4,1428,Hoggi Strollers,HB3232-1000,Bingo Evolution Adaptive Stroller,1,0,45,90,160,19.0,132.0,6.3,17.7,7.0,13.8,0


In [5]:
with open(json_file_path , encoding= 'utf8') as file:
    data = json.load(file)


In [6]:
df = pd.json_normalize(data)

In [7]:
duplicate = df[df.duplicated()]

In [8]:
data = pd.merge(products_online ,  df , left_on= 'Name'  , right_on= 'sku')

In [9]:
data

,Internal ID,BRAND,Name,Store Display Name,All-Terrain tires (K0bby),Transit option (Bus tie-down),Tilt Adjustment,Recline Adjustment Upper,Recline Adjustment Lower,Product Weight,...,Reversible Seating,id,sku,name,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6
0,361,Adaptive Star,ASENDEAVOUR,Axiom Endeavour Mobility Push Chair,1,0,0,0,30,43.0,...,0,c0075795-0521-4fd4-b9ca-5616f9c1af3b,ASENDEAVOUR,Adaptive star axiom endeavour indoor/outdoor mobility push chair,Special Needs Kids,Strollers and Pushchairs,NaN,NaN,NaN,NaN
1,361,Adaptive Star,ASENDEAVOUR,Axiom Endeavour Mobility Push Chair,1,0,0,0,30,43.0,...,0,c0075795-0521-4fd4-b9ca-5616f9c1af3b,ASENDEAVOUR,Adaptive star axiom endeavour indoor/outdoor mobility push chair,More Products,Featured Rehab Brands,Adaptive Star,NaN,NaN,NaN
2,361,Adaptive Star,ASENDEAVOUR,Axiom Endeavour Mobility Push Chair,1,0,0,0,30,43.0,...,0,c0075795-0521-4fd4-b9ca-5616f9c1af3b,ASENDEAVOUR,Adaptive star axiom endeavour indoor/outdoor mobility push chair,More Products,Bids Purchasing Agencies,Strollers And Pushchairs - Bids,NaN,NaN,NaN
3,362,Adaptive Star,ASIMPROV,Axiom Improv Mobility Push Chair,1,0,0,0,30,35.0,...,0,231fe0ad-5be6-4610-8566-e2525df76933,ASIMPROV,Adaptive star axiom improv indoor/outdoor mobility push chair,Special Needs Kids,Strollers and Pushchairs,NaN,NaN,NaN,NaN
4,362,Adaptive Star,ASIMPROV,Axiom Improv Mobility Push Chair,1,0,0,0,30,35.0,...,0,231fe0ad-5be6-4610-8566-e2525df76933,ASIMPROV,Adaptive star axiom improv indoor/outdoor mobility push chair,More Products,Featured Rehab Brands,Adaptive Star,NaN,NaN,NaN
5,362,Adaptive Star,ASIMPROV,Axiom Improv Mobility Push Chair,1,0,0,0,30,35.0,...,0,231fe0ad-5be6-4610-8566-e2525df76933,ASIMPROV,Adaptive star axiom improv indoor/outdoor mobility push chair,More Products,Bids Purchasing Agencies,Strollers And Pushchairs - Bids,NaN,NaN,NaN
6,363,Adaptive Star,ASLASSEN,Axiom Lassen Mobility Push Chair,1,0,0,0,30,35.0,...,0,0e9f92af-2e3c-4b8e-b1dc-d216378bc9ce,ASLASSEN,Adaptive star axiom lassen indoor/outdoor mobility push chair,Special Needs Kids,Strollers and Pushchairs,NaN,NaN,NaN,NaN
7,363,Adaptive Star,ASLASSEN,Axiom Lassen Mobility Push Chair,1,0,0,0,30,35.0,...,0,0e9f92af-2e3c-4b8e-b1dc-d216378bc9ce,ASLASSEN,Adaptive star axiom lassen indoor/outdoor mobility push chair,More Products,Featured Rehab Brands,Adaptive Star,NaN,NaN,NaN
8,363,Adaptive Star,ASLASSEN,Axiom Lassen Mobility Push Chair,1,0,0,0,30,35.0,...,0,0e9f92af-2e3c-4b8e-b1dc-d216378bc9ce,ASLASSEN,Adaptive star axiom lassen indoor/outdoor mobility push chair,More Products,Bids Purchasing Agencies,Strollers And Pushchairs - Bids,NaN,NaN,NaN
9,48671,Adaptive Star,ASPHOENIX,Axiom Phoenix Mobility Push Chair,1,0,0,0,140,30.0,...,0,a2339ff9-6433-497b-bd51-804e2366d21d,ASPHOENIX,Adaptive star axiom phoenix indoor/outdoor mobility push chair,Special Needs Kids,Strollers and Pushchairs,NaN,NaN,NaN,NaN


In [10]:
 data = data.drop(columns=['name' , 'Name'])

In [11]:
#Text Pre-procssing
data['Store Display Name'] = data['Store Display Name'].str.lower()
data['Store Display Name'] = data['Store Display Name'].apply(lambda x: re.sub(r'\W', ' ', x))
data['Category 1'] = data['Category 1'].str.lower()
data['Category 2'] = data['Category 2'].str.lower()
data['Category 3'] = data['Category 3'].str.lower()
data['Category 4'] = data['Category 4'].str.lower()
data['Category 5'] = data['Category 5'].str.lower()
#data['Department'] = data['Department'].str.lower()
#data['Application'] = data['Application'].str.lower()
#data['Application'] = data['Application'].apply(lambda x: re.sub(r'\W', ' ', x))
#data['More filters'] = data['More filters'].str.lower()

In [12]:
#Select Text based features
text_columns = ['Store Display Name','Category 1','Category 2' , 'Category 3','Category 4','Category 5']

In [13]:
#select Numeric Features
numeric_features = ["All-Terrain tires (K0bby)", "Transit option (Bus tie-down)", "Tilt Adjustment"
                    ,"Recline Adjustment Upper","Recline Adjustment Lower"
                   , "Weight capacity"]
#"Seat depth Lower","Seat depth Upper","Seat width Lower"
#                   ,"Seat width Upper","Reversible Seating"

In [14]:
numeric_features = data[numeric_features]

In [15]:
#Calculate Similarity Matrix 
similarity_matrix_features = cosine_similarity(numeric_features, numeric_features)

In [16]:
#Replacing null with N/A
data = data.fillna("N/A")

In [17]:
#initiating Tfid Vectoriser
text_data = data[text_columns].apply(lambda x: ' '.join(x), axis=1)
vectorizer = TfidfVectorizer()
product_matrix = vectorizer.fit_transform(text_data)

In [18]:
#Calculate Similarity Matrix 
cosine_sim = cosine_similarity(product_matrix, product_matrix)

In [19]:
alpha = 0.3  # Weight for similarity based on features
beta =0.7  # Weight for similarity based on product names
similarity_matrix = alpha * cosine_sim + beta * similarity_matrix_features

In [20]:
def get_recommendations(sku, similarity_matrix, products_df):
    index = products_df.index[products_df['sku'] == sku].tolist()[0]
    sim_scores = list(enumerate(similarity_matrix[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[2:7]  # Get top 5 similar products
    product_indices = [i[0] for i in sim_scores]
    return products_df.iloc[product_indices]

In [21]:
sku = "ST62000000"
recommendations = get_recommendations(sku, cosine_sim, data)
print("Recommendations for '{}'".format(sku))
print(recommendations['Store Display Name'])

Recommendations for 'ST62000000'
42       lightning special needs stroller
76        swifty 2 special needs stroller
78          swifty special needs stroller
94            vivo special needs stroller
39    lightning se special needs stroller
Name: Store Display Name, dtype: object


In [22]:
def precision_at_k(actual_items, recommended_items, k):
    relevant_items = set(actual_items)
    recommended_k_items = recommended_items[:k]
    num_relevant_k = len(relevant_items.intersection(recommended_k_items))
    precision = num_relevant_k / k
    return precision

In [23]:
def recall_at_k(actual_items, recommended_items, k):
    relevant_items = set(actual_items)
    recommended_k_items = recommended_items[:k]
    num_relevant_k = len(relevant_items.intersection(recommended_k_items))
    recall = num_relevant_k / len(relevant_items)
    return recall

In [24]:
# ground_truth_items = ['Thomashilfen tRide Special Needs Stroller with T-Chassis ','Thomashilfen tRide Special Needs Stroller with T-Chassis']  # Ground truth items for evaluation
# k = 3  # Top K recommendations

# recommended_items = recommendations['PRODUCT NAME'].tolist()
# precision = precision_at_k(ground_truth_items, recommended_items, k)
# recall = recall_at_k(ground_truth_items, recommended_items, k)

# print("Evaluation Metrics:")
# print("Precision at {}: {:.2f}".format(k, precision))
# print("Recall at {}: {:.2f}".format(k, recall))